# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели *кредитного скоринга* — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — поиск ответов на вопросы:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?



**Описание данных**
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

### Шаг 1. Обзор данных

Перед началом работы импортируем библиотеку `Pandas` и прочитаем нужный файл, предварительно сохранив его в отдельной переменной:

In [35]:
# импорт нужных библиотек
import pandas as pd

# пригодится на этапе категоризации
from pymystem3 import Mystem
m = Mystem()

# чтение файла с данными с сохранением в переменной clients
clients = pd.read_csv('/datasets/data.csv')

Посмотрим, что представляет из себя таблица - выведем первые 25 строк (10 строк - слишком мало, можно не увидеть структуру данных):

In [36]:
# получение первых 25 строк таблицы
clients.head(25)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


По таблице сразу видно, что с данными что-то не так:
1. Есть пропущенные значения;
2. Данные об образовании записаны в разном регистре;
3. В двух столбцах данные имеют значения после запятой (для целей исследования не нужна такая точность);
4. Данные о стаже имеют знак минуса;
5. Одни и те же цели кредита записаны разными способами.

На следующих этапах необходимо устранить эти ошибки, а пока выведем общую статистику по таблице, чтобы поискать другие:

In [37]:
# вывод общей статистики по таблице
clients.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Описательная статистика не подходит для категориальных переменных, поэтому обращаем внимание только на столбцы с количественными данными. В них есть аномалии:

`children`:
* минимальное значение равно -1 (вероятно, данные отсутствуют);
* максимальное значение равно 20 (может, кто-то и правда имеет 20 детей, однако нужно проверить, сколько таких строк всего в таблице - это может быть ошибка);

`days_employed`:
* максимальное значение стажа составляет 1115 лет (401755 дней) - такого не может быть;
* среднее значение показывает, что есть мощные выбросы в данных (люди не могут работать 175 лет);

`dob_years`:
* минимальное значение возраста - 0 лет (вероятно, данные отсутствуют).

Теперь получим общую информацию о таблице командой `.info()`:

In [38]:
# получение общей информации о данных в таблице
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. Кажется, в их названиях нет ошибок: соблюден 'змеиный_регистр', все буквы строчные, пробелы не встречаются. В двух столбцах есть пропуски, которые необходимо заполнить.

### Шаг 2.1 Заполнение пропусков

Посмотрим, сколько всего в таблице пропущенных значений:

In [39]:
# подсчет пропусков
clients.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Для некоторых клиентов не указаны общий трудовой стаж в днях и ежемесячный доход. Вероятно, это связано с непредоставлением справки с места работы. Количество пропусков одинаковое - значит, это одни и те же люди. Так как нет возможности восстановить эти данные, придется заполнить пропуски медианным значением (среднее значение чувствительно к выбросам - значения, которые могут на порядки отличаться от остальных - и поэтому может сильно искажать данные). Однако прежде нужно проверить, из разных ли категорий работников эти люди (чтобы понять, можно ли заполнить NaN медианой по всей группе).

Выполним код:

In [40]:
# проверка принадлежности пропусков к типу занятости
clients[(clients['total_income'].isna() == True) & (clients['days_employed'].isna() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Пропуски есть в разных категориях работников. Будем заполнять их по медиане из каждой группы. Лучше всего заполнять пропуски в `days_employed` медианой по возрасту, а пропуски в в `total_income` - медианой по типу занятости.

In [41]:
# заполнение пропусков в стаже и доходах медианами
clients['days_employed'] =  clients['days_employed'].fillna(clients.groupby('income_type')['days_employed'].transform('median'))
clients['total_income'] =  clients['total_income'].fillna(clients.groupby('dob_years')['total_income'].transform('median'))

Проверим, остались ли в таблице пропуски:

In [42]:
# повторный подсчет пропусков
clients.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Отлично, пропусков нет, можно идти дальше.

### Шаг 2.2 Проверка данных на аномалии и исправления.

На этапе обзора данных были выявлены аномальные данные и исправления. Поэтапно разберемся с ними.

В столбце `education`есть одни и те же значения, записанные с использованием заглавных и строчных букв - необходимо привести их к нижнему регистру.

In [43]:
# приведение значений к нижнему регистру
clients['education'] = clients['education'].str.lower()

# проверка уникальных значений в столбце education
clients['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Теперь все значения имеют один вид.

Столбец `children` содержит странные данные. Выведем информацию о них:

In [44]:
# вывод количества клиентов с детьми
print("Клиенты с -1 ребенком:", clients[clients['children'] == -1].count()[0])
print("Количество строк с 20 детьми:", clients[clients['children'] == 20].count()[0])
print("Количество строк с 0 детьми:", clients[clients['children'] == 0].count()[0])
print("Количество уникальных клиентов с 20 детьми:", len(clients[clients['children'] == 20]['total_income'].unique()))

# проверка уникальных значений количества детей
clients['children'].unique()

Клиенты с -1 ребенком: 47
Количество строк с 20 детьми: 76
Количество строк с 0 детьми: 14149
Количество уникальных клиентов с 20 детьми: 75


array([ 1,  0,  3,  2, -1,  4, 20,  5])

Итак, проверка уникальных значений показывает, что значение 20 сильно выделяется среди остальных - нет клиентов, у которых хотя бы больше 10 детей - это явно ошибочное значение, которое можно заменить на 2. Также заменим -1 на 0. Эти замены несильно испортят данные.

In [45]:
# замена неверных значений количества детей
clients['children'] = clients['children'].replace(-1,0)
clients['children'] = clients['children'].replace(20,2)

# проверка уникальных значений количества детей
clients['children'].unique()

array([1, 0, 3, 2, 4, 5])

С детьми разобрались - теперь все в порядке.

Столбец `days_employed` имеет несколько проблем. Для начала разберемся с отрицательными значениями. Попробуем взять их по модулю, чтобы сделать пригодными для анализа:

In [46]:
# замена отрицательных значений положительными
clients['days_employed'] = abs(clients['days_employed'])

# проверка отрицательных значений
clients['days_employed'].head()

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
Name: days_employed, dtype: float64

Теперь минусы нас не смущают.

Далее выясним, почему в данных есть слишком большие значения.

In [47]:
# вывод среднего значения стажа по категориям работников
clients_mean_grouped = clients.groupby('income_type').agg({'days_employed':'mean'})
clients_mean_grouped

,days_employed
income_type,
безработный,366413.652744
в декрете,3296.759962
госслужащий,3328.308350
компаньон,2055.165652
пенсионер,365025.963652
предприниматель,520.848083
сотрудник,2251.736421
студент,578.751554


Стаж слишком высок в группах безработных и пенсионеров. Предположительно, для них данные указаны в часах, в то время как для всех остальных - в днях.

Максимальное количество отработанных дней является ошибкой.
Напишем функцию, которая будет искать повышенное значение отработанных дней и заменять их на медиану стажа данного возраста.
В случае, если большинство показателей стажа данного возраста завышены, функция будет заменять значение на максимально возможный стаж по возрасту.

In [48]:
# создание функции для обработки ошибок в столбце со стажем 
def err_days_employed(row):
    if row['days_employed'] > (row['dob_years'] - 16) * 365:
        if clients[clients['dob_years'] == row['dob_years']]['days_employed'].median() < (row['dob_years'] - 16) * 365:
            return clients[clients['dob_years'] == row['dob_years']]['days_employed'].median()
        else: 
            return (row['dob_years'] - 16) * 365
    else:
        return row['days_employed']

# применение функции
clients['days_employed'] = clients.apply(err_days_employed, axis=1)

Проверим теперь среднее значение стажа после применения функции:

In [49]:
# проверка среднего значения стажа по категориям работников
clients_mean_grouped = clients.groupby('income_type').agg({'days_employed':'mean'})
clients_mean_grouped

,days_employed
income_type,
безработный,1739.600715
в декрете,3296.759962
госслужащий,3233.166879
компаньон,2006.061018
пенсионер,13126.511186
предприниматель,520.848083
сотрудник,2177.272090
студент,578.751554


Ошибки обработаны. В таблице больше нет клиентов, которые работают дольше, чем живут.

В столбце `dob_years` есть клиенты, которые еще не родились (0 лет). Самое логичное объяснение - возраст просто не указан.

In [50]:
# вывод количества клиентов без указания возраста
print("Количество строк с dob_years = 0:", clients[clients['dob_years'] == 0].count()[0])

Количество строк с dob_years = 0: 101


Заменим пропущенные значения медианой по каждому типу занятости:

In [51]:
# заполнение пропусков медианой
clients['dob_years'] = clients['dob_years'].replace(0, clients.groupby('income_type')['dob_years'].transform('median'))

# проверка количества клиентов без указания возраста
print("Количество строк с dob_years = 0:", clients[clients['dob_years'] == 0].count()[0])

Количество строк с dob_years = 0: 0


Теперь в столбце с возрастом нет нулевых значений.

### Шаг 2.3. Изменение типов данных.

На этапе обзора данных мы увидели, что столбцы `days_employed` и `total_income` имеют значения после запятой. Исправим это и заменим тип на целочисленный:

In [52]:
# замена вещественного типа на целочисленный
clients['days_employed'] = clients['days_employed'].astype('int')
clients['total_income'] = clients['total_income'].astype('int')

# проверка замены типов
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Замена произведена, запятых больше нет.

### Шаг 2.4. Удаление дубликатов.

Мы установили наличие явных дубликатов в столбцах с данными об образовании и целях кредита. Теперь установим полное количество дубликатов и удалим их:

In [53]:
# проверка количества дубликатов
clients.duplicated().sum()

71

Посмотрим на них ближе:

In [54]:
# вывод дубликатов с сортировкой по наиболее вариабельным столбцам
clients[clients.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5865,0,18250,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,111673,операции со своей недвижимостью
9528,0,18250,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,111673,операции со своей недвижимостью
13035,0,17885,65,среднее,1,гражданский брак,1,F,пенсионер,0,122934,сыграть свадьбу
20187,0,17885,65,среднее,1,гражданский брак,1,F,пенсионер,0,122934,сыграть свадьбу
6537,0,20075,71,среднее,1,гражданский брак,1,F,пенсионер,0,122934,на проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
19832,0,1574,48,среднее,1,женат / замужем,0,F,сотрудник,0,158989,ремонт жилью
13712,0,1574,42,среднее,1,женат / замужем,0,F,сотрудник,0,159167,покупка жилья для семьи
20165,0,1574,42,среднее,1,женат / замужем,0,F,сотрудник,0,159167,покупка жилья для семьи
8886,1,1574,37,среднее,1,женат / замужем,0,F,сотрудник,0,160434,покупка недвижимости


Действительно, строки повторяются. Похоже на техническую ошибку, задвоение данных. Можно смело удалять их.

In [55]:
# удаление дубликатов с новой индексацией
clients = clients.drop_duplicates().reset_index(drop=True)

# повторная проверка количества дубликатов
clients.duplicated().sum()

0

Дубликатов не осталось, можно продолжать работу с данными.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

По условию задания создадим два датафрейма с идентификаторами и обозначениями для семейного статуса и уровня образования.

In [58]:
# новый датафрейм для образования
edu_dict = clients[['education', 'education_id']]
edu_dict = edu_dict.drop_duplicates().reset_index(drop=True)
edu_dict.sort_values(by = 'education_id')

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [59]:
# новый датафрейм для семейного статуса
fam_dict = clients[['family_status', 'family_status_id']]
fam_dict = fam_dict.drop_duplicates().reset_index(drop=True)
fam_dict.sort_values(by = 'family_status_id')

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Удалим из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`.

In [60]:
# удаление столбцов
clients = clients.drop(['education', 'family_status'], axis=1)

# проверка удаления
clients.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,3221,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


In [83]:
# добавление словарей в итоговую таблицу для облегчения анализа в последующем
# (на  самом деле, я не знаю, как правильно сделать этот этап)
clients_final = clients.merge(edu_dict, on = 'education_id', how = 'left').merge(fam_dict, on ='family_status_id', how ='left')

clients_final.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,education,family_status
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,высшее,женат / замужем
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,среднее,женат / замужем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,среднее,женат / замужем
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,среднее,женат / замужем
4,0,3221,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,среднее,гражданский брак


### Шаг 2.6. Категоризация дохода.

Значений в столбце `total_income` слишком много, можно объединить их в категории:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [61]:
# создание функции для категоризации дохода
def income_group(income):
    if income <= 30000:
        return 'E'
    if (30001 <= income <= 50000):
        return 'D'
    if (50001 <= income <= 200000):
        return 'C'
    if (200001 <= income <= 1000000):
        return 'B'
    if income >= 100001:
        return 'A'

# создание нового столбца с категориями дохода
clients['total_income_category'] = clients['total_income'].apply(income_group)

# вывод первых строк таблицы
clients.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,3221,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


Теперь все клиенты распределены по группам дохода.

### Шаг 2.7. Категоризация целей кредита.

Для удобства анализа нам нужно создать функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:
* 'операции с автомобилем',
* 'операции с недвижимостью',
* 'проведение свадьбы',
* 'получение образования'.

Например, если в столбце `purpose` находится подстрока 'на покупку автомобиля', то в столбце `purpose_category` должна появиться строка 'операции с автомобилем'.

Сначала изучим данные в стоблце `purpose`:

In [62]:
# вывод всех значений стоблца
clients['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Слишком много одинаковых целей, надо с этим что-то делать. Создадим для них категории:

In [63]:
# создание функции с лемматизацией и возвращением категории
def purpose_type(purpose):
    try:
        lemmas = m.lemmatize(purpose)
        if 'жиль' in purpose:
            return 'операции с недвижимостью'
        if 'недвижимост' in purpose:
            return 'операции с недвижимостью'
        if 'свадьб' in purpose:
            return 'проведение свадьбы'
        if 'автомобил' in purpose:
            return 'операции с автомобилем'
        if 'образовани' in purpose:
            return 'получение образования'
    except:
        return 'без категории'

# добавление нового столбца с категориями
clients['purpose_category'] = clients['purpose'].apply(purpose_type)

# проверка значений и создания категорий
clients['purpose_category'].value_counts()

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

Категории созданы. Проверим, добавился ли новый столбец `purpose_category`:

In [64]:
# вывод первых строк таблицы
clients.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,3221,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


Теперь данные гораздо удобнее анализировать.

### Ответы на вопросы.

В самом начале исследования перед нами стояла задача ответить на следующие вопросы:
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

Теперь у нас есть все, чтобы выполнить эту задачу.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для ответа на этот вопрос нужно создать таблицу с данными о задолженности и количестве детей.

In [65]:
# создание сводной таблицы по количеству детей
clients_children_table = clients.pivot_table(index = ['children'], columns = 'debt', values = 'gender', aggfunc = 'count')

# создание столбца с указанием доли
clients_children_table['ratio_no_debt'] = clients_children_table[0]/len(clients)*100

clients_children_table

debt,0,1,ratio_no_debt
children,,,
0,13074.0,1064.0,60.939685
1,4364.0,444.0,20.341195
2,1926.0,202.0,8.977347
3,303.0,27.0,1.412324
4,37.0,4.0,0.172462
5,9.0,NaN,0.041950


##### Вывод 1:

Как видно по таблице, количество детей заметно влияет на возврат кредита в срок - чем больше детей, тем дольше будет возвращаться кредит. Чаще всего возвращают кредит без задержек люди без детей или с одним ребенком.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создаем таблицу с указанием задолженности и семейного статуса.

In [82]:
# создание сводной таблицы по семейному статусу
clients_family_table = clients_final.pivot_table(index = ['family_status'], columns = 'debt', values = 'gender', aggfunc = 'count')

# создание столбца с указанием доли
clients_family_table['ratio_no_debt'] = clients_family_table[0]/len(clients)*100

clients_family_table

debt,0,1,ratio_no_debt
family_status,,,
Не женат / не замужем,2536,274,11.820640
в разводе,1110,85,5.173860
вдовец / вдова,896,63,4.176377
гражданский брак,3763,388,17.539853
женат / замужем,11408,931,53.174233


##### Вывод 2:

Данные говорят о том, что люди, состоящие в официальном (с большей вероятностью) или гражданском (с меньшей вероятностью) браке, чаще возвращают кредит в положенный срок, чем те, кто в нем не состоит или по каким-либо причинам в него не вступает. Наличие супруга, таким образом, увеличивает доверие к клиенту.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Теперь посмотрим зависимость задолженности от уровня дохода.

In [67]:
# создание сводной таблицы по уровню дохода
clients_income_table = clients.pivot_table(index = ['total_income_category'], columns = 'debt', values = 'gender', aggfunc = 'count')

# создание столбца с указанием доли
clients_income_table['ratio_no_debt'] = clients_income_table[0]/len(clients)*100

clients_income_table

debt,0,1,ratio_no_debt
total_income_category,,,
A,23,2,0.107206
B,4685,356,21.837420
C,14656,1360,68.313601
D,329,21,1.533514
E,20,2,0.093223


##### Вывод 3:

Удивительно, но миллионеры (доход от 1000001, категория А) имеют задолженность чаще, чем средний класс (доход от 50001 до 1000000, категории В и С). Также часто не возвращают кредиты в срок люди, получающие менее 50000 (категории D и E).

Средний класс делится на две группы. С большей вероятностью вернут кредит в срок люди из категории С (доход 50001–200000).

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Последний вопрос, на который снова найдем ответ в сводной таблице.

In [68]:
# создание сводной таблицы по цели кредита
clients_purpose_table = clients.pivot_table(index = ['purpose_category'], columns = 'debt', values = 'gender', aggfunc = 'count')

# создание столбца с указанием доли
clients_purpose_table['ratio_no_debt'] = clients_purpose_table[0]/len(clients)*100

clients_purpose_table

debt,0,1,ratio_no_debt
purpose_category,,,
операции с автомобилем,3903,403,18.192412
операции с недвижимостью,10029,782,46.746527
получение образования,3643,370,16.980516
проведение свадьбы,2138,186,9.965508


##### Вывод 4:

Самая низкая вероятность задолженности у людей, берущих кредит на недвижимость. Вероятно, люди боятся потерять ее при невыплате кредита. Кредиты на образование и покупку автомобиля возвращаются в срок немного чаще, чем кредит на проведение свадьбы.

## Общий вывод:

Полученные данные позволяют нам говорить о том, что есть зависимость между количеством детей, семейным положением, уровнем дохода и задолженностью по кредиту.

В итоге, мы можем сказать, что

Более надежный заемщик:
* Имеет 1 ребенка или не имеет детей вовсе;
* Имеет супруга либо сожителя;
* Принадлежит к среднему классу и получает от 50 тыс. до миллиона рублей;
* Берет кредит на недвижимость;

Менее надежный заемщик:
* Имеет много детей (уже количество детей больше одного увеличивает вероятность задолженности);
* Не состоит в браке по каким-либо причинам;
* Имеет слишком низкий или слишком высокий доход;
* Берет кредит на получение образования, покупку автомобиля или проведение свадьбы.